In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import errno

import CGDs
import importlib
importlib.reload(CGDs)


from pyDOE import lhs
from torch import from_numpy

import torch
import torch.cuda
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad

# Manage device

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
# torch.set_default_tensor_type(torch.DoubleTensor)
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


In [3]:
def printMemory():
  t = torch.cuda.get_device_properties(0).total_memory
  r = torch.cuda.memory_reserved(0)
  a = torch.cuda.memory_allocated(0)
  f = r-a  # free inside reserved
  print(f"total: {t}, reserved: {r}, free: {f}")
  
printMemory()

total: 25756696576, reserved: 0, free: 0


# Create Neural Network Module

In [4]:
import networks
from networks import *
layers = np.array([2, 50, 50, 50, 1])
#(self, layers, x_test, y_test, u_test, x_bc, y_bc, u_bc, fxy, x_inside_train, y_inside_train):
# Create the model
PINNBCGD = networks.FeedforwardNeuralNetwork(layers[0], layers[1], layers[-1], len(layers)-1)
PINNBCGD.to(device)
print(PINNBCGD)

FeedforwardNeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1-3): 3 x Linear(in_features=50, out_features=50, bias=True)
  )
  (output): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)


In [5]:
DGM = networks.DGMCell(layers[0], layers[1], layers[-1], len(layers)-1)
DGM.to(device)
print(DGM)

DGMCell(
  (sig_act): Tanh()
  (Sw): Linear(in_features=2, out_features=50, bias=True)
  (Uz): Linear(in_features=2, out_features=50, bias=True)
  (Wsz): Linear(in_features=50, out_features=50, bias=True)
  (Ug): Linear(in_features=2, out_features=50, bias=True)
  (Wsg): Linear(in_features=50, out_features=50, bias=True)
  (Ur): Linear(in_features=2, out_features=50, bias=True)
  (Wsr): Linear(in_features=50, out_features=50, bias=True)
  (Uh): Linear(in_features=2, out_features=50, bias=True)
  (Wsh): Linear(in_features=50, out_features=50, bias=True)
  (Wf): Linear(in_features=50, out_features=1, bias=True)
)


In [6]:
D_BCGD = Discriminator(2, 25 ,2)
D_BCGD.to(device)
D_BCGD.load_state_dict(D_BCGD.state_dict()) # copy weights and stuff
print(D_BCGD)

Discriminator(
  (map): Sequential(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): ReLU()
    (8): Linear(in_features=50, out_features=2, bias=True)
  )
)


# Network Trainig

### Data sampling

In [7]:
import utils
from utils import *

samples = {"pde": 50000, "bc":5000, "fc":5000}

K = 40.0
r = 0.05
sigma = 0.25
T = 1.0
S_range = [0.0, 130.0]
t_range = [0.0, T]
gs = lambda x: np.fmax(x-K, 0.0)

In [8]:
all_st_train, bc_st_train, bc_v_train, n_st_train, n_v_train = utils.trainingData(K, 
                                                                                  r, 
                                                                                  sigma, 
                                                                                  T, 
                                                                                  S_range[-1], 
                                                                                  S_range, 
                                                                                  t_range, 
                                                                                  gs, 
                                                                                  samples['bc'], 
                                                                                  samples['fc'], 
                                                                                  samples['pde'], 
                                                                                  RNG_key=123
                                                                                  )

# a1, a2 = get_diff_data(samples['pde'])
# b1, b2, c1, c2 = get_bvp_data(samples['bvp'])
# d1, d2 = get_fvp_data(samples['fvp'])

# fig = plt.figure(figsize=(9,6))
# plt.scatter([sublist[0] for sublist in a1], [sublist[1] for sublist in a1], marker='.',alpha=0.3)
# plt.scatter([sublist[0] for sublist in b1], [sublist[1] for sublist in b1], marker='X')
# plt.scatter([sublist[0] for sublist in c1], [sublist[1] for sublist in c1], marker='X')
# plt.scatter([sublist[0] for sublist in d1], [sublist[1] for sublist in d1], marker='X')
# plt.xlabel('Time t')
# plt.ylabel('Option Price s')

# plt.title('Positions of collocation points and boundary data');

### Hyperparameters

In [9]:
max_iter = 60001
graphPer = 0
iter_recordBCGD = 200

savePer = 60001

# Define loss function and optimizer
optimizer = CGDs.BCGD(max_params=D_BCGD.parameters(), min_params= PINNBCGD.parameters(), device = device,
                 lr_max=0.02, lr_min=0.02, tol=1e-10, collect_info=True)
lossFunction = torch.nn.MSELoss()

### Training

In [10]:
start_time=time.time()
loss_hist = []

for epoch in range(max_iter):
    
    all_st_train, bc_st_train, bc_v_train, n_st_train, n_v_train = \
    utils.trainingData(K, 
                       r, 
                       sigma, 
                       T, 
                       S_range[-1], 
                       S_range, 
                       t_range, 
                       gs, 
                       samples['bc'], 
                       samples['fc'], 
                       samples['pde'], 
                       RNG_key=123)
    
    optimizer.zero_grad()
    
    # normal loss
    # print(n_st_train)
    # print(PINNBCGD.output.weight.dtype)
    v1_hat = PINNBCGD(n_st_train)
    
    grads = tgrad.grad(v1_hat, n_st_train, grad_outputs=torch.ones(v1_hat.shape).cuda(), 
                       retain_graph=True, create_graph=True, only_inputs=True)[0]
    dVdt, dVdS = grads[:, 0].view(-1, 1), grads[:, 1].view(-1, 1)
    grads2nd = tgrad.grad(dVdS, n_st_train, grad_outputs=torch.ones(dVdS.shape).cuda(), create_graph=True, only_inputs=True)[0]
    d2VdS2 = grads2nd[:, 1].view(-1, 1)
    S1 = n_st_train[:, 1].view(-1, 1)
    pde_loss = lossFunction(-dVdt, 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*v1_hat)
    
    D_output1 = D_BCGD(n_st_train[:, [0]], n_st_train[:, [1]])
    loss1 = D_output1[:,[0]] * pde_loss
    
    
    # boundary condition loss
    bc_hat = PINNBCGD(bc_st_train)
    bc_loss = lossFunction(bc_v_train, bc_hat)
    
    loss2 = D_BCGD(bc_st_train[:, [0]], bc_st_train[:, [0]])[:,[1]] * (PINNBCGD(bc_st_train) - bc_v_train)
    
    
    
    # Backpropagation and Update
    combined_loss = loss1.mean() + loss2.mean()
    # combined_loss.backward()
    optimizer.step(combined_loss)
    
    loss_hist.append(combined_loss.item())
    if epoch % 500 == 0:
        print(f'{epoch}/{max_iter} PDE Loss: {pde_loss.item():.5f}, BC Loss: {bc_loss.item():.5f}')
        pass
        
        
print('finish')

0/60001 PDE Loss: 0.00003, BC Loss: 3454.94995
